In [13]:
from common import *
import numpy as np
import time

In [16]:
board = initialize_game_state()
board = apply_player_action(board, 4, PLAYER1)
board = apply_player_action(board, 4, PLAYER2)
board = apply_player_action(board, 3, PLAYER1)
board = apply_player_action(board, 2, PLAYER2)
board1 = apply_player_action(board, 5, PLAYER1)
board2 = apply_player_action(board1, 2, PLAYER2)

board2

array([[ 0,  0, -1,  1,  1,  1,  0],
       [ 0,  0, -1,  0, -1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0]], dtype=int8)

In [ ]:
ITERATIONS = 1000
player = PLAYER1

In [38]:
class Node():
    
    def __init__(self, player, play, board, parent=None):
        self.board = board
        self.player = player  # 1 for maximizer or -1 for minimizer
        self.parent = parent
        self.play = play    
        self.children = {}

        
        if is_terminal_board(self.board, self.player):
            self.is_terminal = True
        else:
            self.is_terminal = False  
        self.is_leaf = self.is_terminal
        
        self.visits = 0
        self.total_score = 0
    
    def valid_actions(self):
        return np.where(self.board[-1, :] == NO_PLAYER)[0] 
        
    
class MCTS():
    
    def __init__(self, iterations, exploration = 1/np.sqrt(2)):
        self.iterations = iterations
        self.exploration = exploration

    def search(self, root_node):
        for iteration in self.iterations:
            pass
        
    def select(self, node):
        while not node.is_terminal:
            if not node.is_leaf:
                node = self.get_best_move()
            else:
                node = self.expand(node)
        return node
        
    def expand(self, node):
        actions = node.valid_actions()
        
        
    def simulate(self, node):
        while not node.is_terminal:
            action = np.random.choice(node.get_valid_actions())
            child = apply_player_action(node.board, action, player)
            
            player *= -1
        
    
    def backpropogate(node, score):
        while node is not None:
            node.visits += 1
            node.total_score += score
            node = node.parent
    
    def get_best_move(self, node, exploration):
        best_score = np.NINF
        best_children = []
        
        for child_node in node.children.values():        
            move_score = child_node.player * child_node.score / child_node.visits + self.exploration * np.sqrt(np.log(child_node.visits/child_node.visits))
        
        if move_score > best_score:
            best_score = move_score
            best_child = [child_node]
        elif move_score == best_score:
            best_child.append(child_node)
            
        return np.random.choice(best_children)
    
  

def mcts(board: np.ndarray, player: BoardPiece, saved_state: Optional[SavedState]) -> Tuple[PlayerAction, Optional[SavedState]]:
  
    valid_columns = np.where(board[-1,:] == NO_PLAYER)[0]
    action = np.random.choice(valid_columns)

    return action, saved_state

In [44]:
node_test = Node(board1, board, PLAYER1)

In [37]:
no

0

AttributeError: 'Node' object has no attribute 'is_root'